<div style="display: flex; background-color: #3F579F;">
    <h1 style="margin: auto; font-weight: bold; padding: 30px 30px 0px 30px;" align="center">Consumption needs of buildings - Project 4</h1>
</div>
<div style="display: flex; background-color: #3F579F; margin: auto; padding: 5px 30px 0px 30px;" >
    <h2 style="width: 100%; text-align: center; float: left;" align="center">| Analysis notebook |</h2>
</div>
<div style="display: flex; background-color: #3F579F; margin: auto; padding: 10px 30px 30px 30px;">
    <h3 style="width: 100%; text-align: center; font-size:26px; float: left;" align="center">Data Scientist course - OpenClassrooms</h3>
</div>

<div style="background-color: #506AB9;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">1. Libraries and functions</h3>
</div>

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">1.1. Libraries and functions</h3>
</div>

In [1]:
import io
import gc
import ast
from math import prod
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from sklearn.decomposition import PCA


# import os
# import itertools
# import math
# import matplotlib.gridspec as gridspec
# from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
# from sklearn.model_selection import train_test_split
# from sklearn.impute import KNNImputer
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.linear_model import LinearRegression



%matplotlib inline
sns.set_theme(style="darkgrid")

SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">1.2. Functions declaration</h4>
</div>

In [2]:
def df_analysis(df, name_df, *args, **kwargs):
    """
    Method used to analyze on the DataFrame.

    Parameters:
    -----------------
        df (pandas.DataFrame): Dataset to analyze
        name_df (str): Dataset name
        
        *args, **kwargs:
        -----------------
            columns (list): Dataframe keys in list format
            flag (str): Flag to show complete information about the dataset to analyse
                        "complete" shows all information about the dataset

    Returns:
    -----------------
        None. 
        Print the analysis on the Dataset. 
    """
    
    # Getting the variables
    columns = kwargs.get("columns", None)
    type_analysis = kwargs.get("type_analysis", None)
    
    ORDERING_COMPLETE = [
        "name", "type", "records", "unique", "# NaN", "% NaN", "mean", "min", "25%", "50%", "75%", "max", "std"
    ]
    
    # Calculating the memory usage based on dataframe.info()
    buf = io.StringIO()
    df.info(buf=buf)
    memory_usage = buf.getvalue().split('\n')[-2]
    
    if df.empty:
        print("The", name_df, "dataset is empty. Please verify the file.")
    else:
        empty_cols = [col for col in df.columns if df[col].isna().all()] # identifying empty columns
        df_rows_duplicates = df[df.duplicated()] #identifying full duplicates rows
        
        # Creating a dataset based on Type object and records by columns
        type_cols = df.dtypes.apply(lambda x: x.name).to_dict() 
        df_resume = pd.DataFrame(list(type_cols.items()), columns = ["name", "type"])
        df_resume["records"] = list(df.count())
        df_resume["# NaN"] = list(df.isnull().sum())
        df_resume["% NaN"] = list(((df.isnull().sum() / len(df.index))*100).round(2))
        
        print("\nAnalysis of", name_df, "dataset")
        print("--------------------------------------------------------------------")
        print("- Dataset shape:                 ", df.shape[0], "rows and", df.shape[1], "columns")
        print("- Total of NaN values:           ", df.isna().sum().sum())
        print("- Percentage of NaN:             ", round((df.isna().sum().sum() / prod(df.shape)) * 100, 2), "%")
        print("- Total of full duplicates rows: ", df_rows_duplicates.shape[0])
        print("- Total of empty rows:           ", df.shape[0] - df.dropna(axis="rows", how="all").shape[0]) if df.dropna(axis="rows", how="all").shape[0] < df.shape[0] else \
                    print("- Total of empty rows:            0")
        print("- Total of empty columns:        ", len(empty_cols))
        print("  + The empty column is:         ", empty_cols) if len(empty_cols) == 1 else \
                    print("  + The empty column are:         ", empty_cols) if len(empty_cols) >= 1 else None
        print("- Unique indexes:                ", df.index.is_unique)
        
        if columns is not None:
            print("\n- The key(s):", columns, "is not present multiple times in the dataframe.\n  It CAN be used as a primary key.") if df.size == df.drop_duplicates(columns).size else \
                print("\n- The key(s):", columns, "is present multiple times in the dataframe.\n  It CANNOT be used as a primary key.")
        
        if type_analysis == "summarized":
            print("\n")
        
        if type_analysis is None or type_analysis != "summarized":
            pd.set_option("display.max_rows", None) # show full of showing rows
            pd.set_option("display.max_columns", None) # show full of showing cols
            pd.set_option("display.max_colwidth", None) # show full width of showing cols
            pd.set_option("display.float_format", lambda x: "%.5f" % x) # show full content in cell    
            
            if type_analysis is None or type_analysis != "complete":
                print("\n- Type object and records by columns      (",memory_usage,")")
                print("--------------------------------------------------------------------")
            elif type_analysis == "complete":
                df_resume["unique"] = list(df.nunique())
                df_desc = pd.DataFrame(df.describe().T).reset_index()
                df_desc = df_desc.rename(columns={"index": "name"})
                df_resume = df_resume.merge(right=df_desc[["name", "mean", "min", "25%", "50%", "75%", "max", "std"]], on="name", how="left")
                df_resume = df_resume[ORDERING_COMPLETE]
                print("\n- Type object and records by columns      (",memory_usage,")")
                print("--------------------------------------------------------------------")
                
            display(df_resume.sort_values("records", ascending=False))
            
            pd.reset_option("display.max_rows") # reset max of showing rows
            pd.reset_option("display.max_columns") # reset max of showing cols
            pd.reset_option("display.max_colwidth") # reset width of showing cols
            pd.reset_option("display.float_format") # reset show full content in cell
            
        # deleting dataframe to free memory
        if type_analysis == "complete":
            del [[df_resume, df_desc]]
            gc.collect()
            df_resume, df_desc = (pd.DataFrame() for i in range(2))
        else:
            del df_resume
            gc.collect()
            df_resume = pd.DataFrame()


def comparing_columns_df():
    """
    Method used to identify columns that do not match between datasets

    Returns:
    -----------------
        df (pandas.DataFrame): Dataframe with columns that do not match
    """

    LIST_COL, LIST_ORIGIN = ([] for i in range(2))

    for col in data_2015.columns:
        if col not in list(data_2016.columns):
            LIST_COL.append(col)
            LIST_ORIGIN.append("data_2015")

    for col in data_2016.columns:
        if col not in list(data_2015.columns):
            LIST_COL.append(col)
            LIST_ORIGIN.append("data_2016")

    df = pd.DataFrame({
        "column": LIST_COL,
        "origin": LIST_ORIGIN
    })
    
    display(df.sort_values("column"))
    
    return df


def types_different():
    """
    Method used to identify the same columns with different dtypes

    Returns:
    -----------------
        None. 
        Print the tests on a new Dataset. 
    """
    
    COLS_2015, COLS_2016, LIST_COL_DIFFERENT, LIST_DTYPE_2015, LIST_DTYPE_2016 = [[] for i in range(5)]
    
    for col in data_2015.columns:
        if col not in list(df_columns_not_matches.loc[df_columns_not_matches["origin"]=="data_2015", "column"]):         
            COLS_2015.append(col)
    for col in data_2016.columns:
        if col not in list(df_columns_not_matches.loc[df_columns_not_matches["origin"]=="data_2016", "column"]):
            COLS_2016.append(col)
    
    for col_2015, col_2016 in zip(COLS_2015, COLS_2016):
        if data_2015[col_2015].dtype != data_2016[col_2016].dtype:
            LIST_COL_DIFFERENT.append(col_2015)
            LIST_DTYPE_2015.append(str(data_2015[col_2015].dtype))
            LIST_DTYPE_2016.append(str(data_2016[col_2016].dtype))
            
    df_types_different = pd.DataFrame({
                            "column": LIST_COL_DIFFERENT,
                            "2015": LIST_DTYPE_2015,
                            "2016": LIST_DTYPE_2016
                        })
    
    return display(df_types_different.sort_values("column"))
            
            
def normality_test(df):
    """
    Method used to make the normality test.

    Parameters:
    -----------------
        df (pandas.DataFrame): Dataset to analyze

    Returns:
    -----------------
        None. 
        Print the tests on a new Dataset. 
    """
    
    list_test = {
        "Shapiro-Wilk":stats.shapiro, "D’Agostino’s K^2":stats.normaltest,
        "Kolmogorov-Smirnov":stats.kstest
    }
    
    
    alpha = 0.05
    fail_to_reject_H = "Sample looks Gaussian (fail to reject H0)"
    reject_H = "Sample does not look Gaussian (reject H0)"
    
    variable, test_name, result, hypothesis = [[] for i in range(4)]
    
    for key, value in list_test.items():
    
        for col in df.columns:
            
            if df[col].dtypes == "float64":
                variable.append(col)
                test_name.append(key)
                
                if key == "Kolmogorov-Smirnov":
                    stat, p_value = value(df[col], cdf="norm")
                else:
                    stat, p_value = value(df[col])
                    
                result.append("Statistics=%.3f, p-value=%.3f" % (stat, p_value))
                hypothesis.append(fail_to_reject_H.translate(SUB)) if p_value > alpha else hypothesis.append(reject_H.translate(SUB))
                
    df_normality_test = pd.DataFrame({
                            "variable": variable,
                            "normality test": test_name, 
                            "result": result,
                            "hypothesis": hypothesis})
    
    display(df_normality_test)

<div style="background-color: #506AB9;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">2. Importing files</h3>
</div>

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">2.1. Importing and preparing files</h4>
</div>

<div class="alert alert-block alert-info">
    Based on the <b>official site of Seattle Open Data</b>, we can see the dataset <b>metadata</b> to know all features
    
    <ul style="list-style-type: square;">
        <li><b>2015: https://data.seattle.gov/dataset/2015-Building-Energy-Benchmarking/h7rm-fz6m</b></li>
        <li><b>2016: https://data.seattle.gov/dataset/2016-Building-Energy-Benchmarking/2bpz-gwpy</b></li>
    </ul>  
    
    On the other hand, we get the means of <b>Energy Star Score</b>
    <ul style="list-style-type: square;">
        <li><b>Energy Star Score: https://www.energystar.gov/buildings/benchmark/analyze_benchmarking_results</b></li>
    </ul>  
</div>

In [3]:
data_2015 = pd.read_csv(r"datasets\2015-building-energy-benchmarking.csv")
data_2016 = pd.read_csv(r"datasets\2016-building-energy-benchmarking.csv")

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">2.2. Initial analysis</h4>
</div>

<div class="alert alert-block alert-info">
Making <b>the initial analysis</b>
</div>

In [4]:
df_analysis(data_2015, "data_2015")


Analysis of data_2015 dataset
--------------------------------------------------------------------
- Dataset shape:                  3340 rows and 47 columns
- Total of NaN values:            26512
- Percentage of NaN:              16.89 %
- Total of full duplicates rows:  0
- Total of empty rows:            0
- Total of empty columns:         0
- Unique indexes:                 True

- Type object and records by columns      ( memory usage: 1.2+ MB )
--------------------------------------------------------------------


,name,type,records,# NaN,% NaN
0,OSEBuildingID,int64,3340,0,0.00000
9,YearBuilt,int64,3340,0,0.00000
40,ComplianceStatus,object,3340,0,0.00000
1,DataYear,int64,3340,0,0.00000
14,PropertyGFABuilding(s),int64,3340,0,0.00000
13,PropertyGFAParking,int64,3340,0,0.00000
12,PropertyGFATotal,int64,3340,0,0.00000
10,NumberofBuildings,int64,3340,0,0.00000
46,Zip Codes,int64,3340,0,0.00000
8,Neighborhood,object,3340,0,0.00000


In [5]:
df_analysis(data_2016, "data_2016")


Analysis of data_2016 dataset
--------------------------------------------------------------------
- Dataset shape:                  3376 rows and 46 columns
- Total of NaN values:            19952
- Percentage of NaN:              12.85 %
- Total of full duplicates rows:  0
- Total of empty rows:            0
- Total of empty columns:         1
  + The empty column is:          ['Comments']
- Unique indexes:                 True

- Type object and records by columns      ( memory usage: 1.2+ MB )
--------------------------------------------------------------------


,name,type,records,# NaN,% NaN
0,OSEBuildingID,int64,3376,0,0.00000
11,Neighborhood,object,3376,0,0.00000
1,DataYear,int64,3376,0,0.00000
40,DefaultData,bool,3376,0,0.00000
42,ComplianceStatus,object,3376,0,0.00000
19,PropertyGFABuilding(s),int64,3376,0,0.00000
18,PropertyGFAParking,int64,3376,0,0.00000
17,PropertyGFATotal,int64,3376,0,0.00000
14,YearBuilt,int64,3376,0,0.00000
13,Longitude,float64,3376,0,0.00000


<div style="background-color: #506AB9;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">3. Cleaning</h3>
</div>

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">3.1. Formatting columns in both datasets</h4>
</div>

<div class="alert alert-block alert-info">
    We are going to <b>concatenate both datasets</b> and keeping only the <b>unique values</b>. But first, we must know deeply the datastes.
</div>
<div class="alert alert-block alert-info">
    Based on the <b>official site of Seattle Open Data</b>, we can <b>identify similar features</b> between datasets
    
     <ul style="list-style-type: square;">
        <li><b>GHGEmissions(MetricTonsCO2e)</b> in <b>data_2015</b> is the same feature as <b>TotalGHGEmissions</b> in <b>data_2016</b></li>
        <li><b>GHGEmissionsIntensity(kgCO2e/ft2)</b> in <b>data_2015</b> is the same feature as <b>GHGEmissionsIntensity</b> in <b>data_2016</b></li>
    </ul>
    
     On the other hands, we have the following
    <ul style="list-style-type: square;">
        <li><b>Zip Codes</b> in <b>data_2015</b> is the same feature as <b>ZipCode</b> in <b>data_2016</b></li>
    </ul>
</div>

In [6]:
data_2015["ZipCode"] = data_2015["Zip Codes"]
data_2016["GHGEmissionsIntensity(kgCO2e/ft2)"] = data_2016["GHGEmissionsIntensity"]
data_2016["GHGEmissions(MetricTonsCO2e)"] = data_2016["TotalGHGEmissions"]

data_2015.drop(labels=["Zip Codes"], axis=1, inplace=True)
data_2016.drop(labels=["GHGEmissionsIntensity", "TotalGHGEmissions"], axis=1, inplace=True)

<div class="alert alert-block alert-info">
    Splitting the featured <b>Location</b> in <b>data_2015</b>
</div>

In [7]:
df_temp = data_2015["Location"].apply(lambda x: ast.literal_eval(x))
data_2015[["Latitude", "Longitude", "HumanAddress"]] = df_temp.apply(pd.Series)

<div class="alert alert-block alert-info">
<b>Comparing</b> the match between columns <b>before</b> making the <b>concatenation</b>
</div>

In [8]:
df_columns_not_matches = comparing_columns_df()

,column,origin
3,2010 Census Tracts,data_2015
8,Address,data_2016
9,City,data_2016
5,City Council Districts,data_2015
2,Comment,data_2015
11,Comments,data_2016
7,HumanAddress,data_2015
0,Location,data_2015
1,OtherFuelUse(kBtu),data_2015
6,SPD Beats,data_2015


<div class="alert alert-block alert-success">
    At this point, we get all columns that do <b>not match between the datasets</b>. Most of them seem not to have <b>importance</b> unlike <b>OtherFuelUse(kBtu)</b>
</div>

<div class="alert alert-block alert-info">
    <b>Eliminate</b> the columns that do not matched between the two datasets because they are not interesting for the mission<br>
    But, keeping the column <b>OtherFuelUse(kBtu)</b>
</div>

In [9]:
COLUMNS_NOT_MACTCH_2015 = list(df_columns_not_matches.loc[(df_columns_not_matches["origin"]=="data_2015") & (df_columns_not_matches["column"]!="OtherFuelUse(kBtu)"), "column"])
COLUMNS_NOT_MACTCH_2016 = list(df_columns_not_matches.loc[df_columns_not_matches["origin"]=="data_2016", "column"])

In [10]:
data_2015.drop(labels=COLUMNS_NOT_MACTCH_2015, axis=1, inplace=True)
data_2016.drop(labels=COLUMNS_NOT_MACTCH_2016, axis=1, inplace=True)

In [11]:
df_columns_not_matches = comparing_columns_df()

,column,origin
0,OtherFuelUse(kBtu),data_2015


<div class="alert alert-block alert-info">
<b>Comparing</b> the types of each par of column <b>before</b> making the <b>concatenation</b>
    
</div>

In [12]:
data_2015 = data_2015.reindex(sorted(data_2015.columns), axis=1)
data_2016 = data_2016.reindex(sorted(data_2016.columns), axis=1)

In [13]:
types_different()

,column,2015,2016
0,DefaultData,object,bool
1,Latitude,object,float64
2,Longitude,object,float64
3,NumberofBuildings,int64,float64
4,NumberofFloors,float64,int64
5,ZipCode,int64,float64


<div class="alert alert-block alert-info">
    <b>Fixing column types</b><br>
</div>

In [14]:
data_2015["DefaultData"] = data_2015["DefaultData"].astype("bool")
data_2015["Latitude"] = data_2015["Latitude"].astype("float64")
data_2015["Longitude"] = data_2015["Longitude"].astype("float64")

data_2016["NumberofBuildings"] = data_2016["NumberofBuildings"].fillna(0)
data_2016["NumberofBuildings"] = data_2016["NumberofBuildings"].astype("int64")

data_2015["NumberofFloors"] = data_2015["NumberofFloors"].fillna(0)
data_2015["NumberofFloors"] = data_2015["NumberofFloors"].astype("int64")

data_2016["ZipCode"] = data_2016["ZipCode"].fillna(0)
data_2016["ZipCode"] = data_2016["ZipCode"].astype("int64")

types_different()

,column,2015,2016


<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">3.2. Merging datasets considering duplicate buildings</h4>
</div>

<div class="alert alert-block alert-info">
    It is necessary to <b>identify</b> those buildings <b>duplicates and not</b> between both datasets
</div>

In [15]:
in_common = data_2015.merge(data_2016, on="OSEBuildingID")
in_2015 = data_2015[~data_2015["OSEBuildingID"].isin(data_2016["OSEBuildingID"])]
in_2016 = data_2016[~data_2016["OSEBuildingID"].isin(data_2015["OSEBuildingID"])]

In [16]:
in_common.shape[0]

3284

In [17]:
in_2015.shape[0]

56

In [18]:
in_2016.shape[0]

92

<div class="alert alert-block alert-info">
    We got the following:
    <ul style="list-style-type: square;">
        <li>There are <b>3284 duplicates between both datasets</b></li>
        <li>There are <b>56 buildings only in data_2015</b></li>
        <li>There are <b>92 buildings only in data_2016</b></li>
    </ul>
    So, we are going to treat the data as follows:
    <ul style="list-style-type: square;">
        <li>Selecting most <b>recent value</b> by <b>Year</b> for each <b>OSEBuildingID</b></li>
        <li>Concatenating <b>data that is not in both datasets</b></li>
    </ul>
</div>

In [19]:
data_temp = pd.concat([data_2015[data_2015["OSEBuildingID"].isin(data_2016["OSEBuildingID"])],\
                      data_2016[data_2016["OSEBuildingID"].isin(data_2015["OSEBuildingID"])]],\
                      ignore_index=True).reset_index(drop=True)

In [20]:
data_temp = data_temp.sort_values(["OSEBuildingID", "DataYear"], ascending=True).reset_index(drop=True)

In [21]:
data_temp.shape

(6568, 43)

In [22]:
pd.set_option("display.max_rows", None) # show full of showing rows
pd.set_option("display.max_columns", None) # show full of showing cols
pd.set_option("display.max_colwidth", None) # show full width of showing cols
pd.set_option("display.float_format", lambda x: "%.5f" % x) # show full content in cell 

In [23]:
data_temp.head(6)

,BuildingType,ComplianceStatus,CouncilDistrictCode,DataYear,DefaultData,ENERGYSTARScore,Electricity(kBtu),Electricity(kWh),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),LargestPropertyUseType,LargestPropertyUseTypeGFA,Latitude,ListOfAllPropertyUseTypes,Longitude,NaturalGas(kBtu),NaturalGas(therms),Neighborhood,NumberofBuildings,NumberofFloors,OSEBuildingID,OtherFuelUse(kBtu),Outlier,PrimaryPropertyType,PropertyGFABuilding(s),PropertyGFAParking,PropertyGFATotal,PropertyName,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SteamUse(kBtu),TaxParcelIdentificationNumber,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearBuilt,YearsENERGYSTARCertified,ZipCode
0,NonResidential,Compliant,7,2015,True,65.00000,3686160.00000,1080307.00000,249.43000,2.64000,Hotel,88434.00000,47.61219,Hotel,-122.33800,1272388.00000,12724.00000,DOWNTOWN,1,12,1,0.00000,NaN,Hotel,88434,0,88434,MAYFLOWER PARK HOTEL,NaN,NaN,78.90000,80.30000,6981428.00000,7097539.00000,173.50000,175.10000,2023032.00000,659000030,NaN,NaN,1927,NaN,18081
1,NonResidential,Compliant,7,2016,False,60.00000,3946027.00000,1156514.25000,249.98000,2.83000,Hotel,88434.00000,47.61220,Hotel,-122.33799,1276453.00000,12764.52930,DOWNTOWN,1,12,1,NaN,NaN,Hotel,88434,0,88434,Mayflower park hotel,NaN,NaN,81.70000,84.30000,7226362.50000,7456910.00000,182.50000,189.00000,2003882.00000,0659000030,NaN,NaN,1927,NaN,98101
2,NonResidential,Compliant,7,2015,True,51.00000,3905411.00000,1144563.00000,263.51000,2.38000,Hotel,83880.00000,47.61311,"Hotel, Parking, Restaurant",-122.33336,4448985.00000,44490.00000,DOWNTOWN,1,11,2,0.00000,NaN,Hotel,88502,15064,103566,PARAMOUNT HOTEL,Parking,15064.00000,94.40000,99.00000,8354235.00000,8765788.00000,191.30000,195.20000,0.00000,659000220,Restaurant,4622.00000,1996,NaN,18081
3,NonResidential,Compliant,7,2016,False,61.00000,3242851.00000,950425.18750,295.86000,2.86000,Hotel,83880.00000,47.61317,"Hotel, Parking, Restaurant",-122.33393,5145082.00000,51450.81641,DOWNTOWN,1,11,2,NaN,NaN,Hotel,88502,15064,103566,Paramount Hotel,Parking,15064.00000,94.80000,97.90000,8387933.00000,8664479.00000,176.10001,179.39999,0.00000,0659000220,Restaurant,4622.00000,1996,NaN,98101
4,NonResidential,Compliant,7,2015,True,18.00000,49762435.00000,14583930.00000,2061.48000,1.92000,Hotel,757243.00000,47.61335,"Hotel, Parking, Swimming Pool",-122.33770,3709900.00000,37099.00000,DOWNTOWN,1,41,3,0.00000,NaN,Hotel,961990,0,961990,WESTIN HOTEL,Parking,100000.00000,96.60000,99.70000,73130656.00000,75506272.00000,242.70000,246.50000,19660404.00000,659000475,Swimming Pool,0.00000,1969,NaN,18081
5,NonResidential,Compliant,7,2016,False,43.00000,49526664.00000,14515435.00000,2089.28000,2.19000,Hotel,756493.00000,47.61393,Hotel,-122.33810,1493800.00000,14938.00000,DOWNTOWN,1,41,3,NaN,NaN,Hotel,759392,196718,956110,5673-The Westin Seattle,NaN,NaN,96.00000,97.70000,72587024.00000,73937112.00000,241.89999,244.10001,21566554.00000,0659000475,NaN,NaN,1969,NaN,98101


<div class="alert alert-block alert-info">
    Selecting most <b>recent value</b> by <b>Year</b> for each <b>OSEBuildingID</b>
</div>

In [24]:
data = data_temp.groupby("OSEBuildingID", as_index=False).last()

In [25]:
data.shape

(3284, 43)

In [26]:
df_analysis(data, "data", columns="OSEBuildingID", type_analysis="summarized")


Analysis of data dataset
--------------------------------------------------------------------
- Dataset shape:                  3284 rows and 43 columns
- Total of NaN values:            15849
- Percentage of NaN:              11.22 %
- Total of full duplicates rows:  0
- Total of empty rows:            0
- Total of empty columns:         0
- Unique indexes:                 True

- The key(s): OSEBuildingID is not present multiple times in the dataframe.
  It CAN be used as a primary key.




<div class="alert alert-block alert-info">
    Concatenating <b>data that is not in both datasets</b>
</div>

In [27]:
data = pd.concat([data, in_2015, in_2016], ignore_index=True).reset_index(drop=True)

In [28]:
df_analysis(data, "data", columns="OSEBuildingID", type_analysis="summarized")


Analysis of data dataset
--------------------------------------------------------------------
- Dataset shape:                  3432 rows and 43 columns
- Total of NaN values:            16761
- Percentage of NaN:              11.36 %
- Total of full duplicates rows:  0
- Total of empty rows:            0
- Total of empty columns:         0
- Unique indexes:                 True

- The key(s): OSEBuildingID is not present multiple times in the dataframe.
  It CAN be used as a primary key.




<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">3.3. Identifying and formatting categorical columns</h4>
</div>

<div class="alert alert-block alert-info">
    List of <b>categorical</b> columns
</div>

In [29]:
list(set(data.columns) - set(data._get_numeric_data()))

['LargestPropertyUseType',
 'ComplianceStatus',
 'ThirdLargestPropertyUseType',
 'ListOfAllPropertyUseTypes',
 'SecondLargestPropertyUseType',
 'PropertyName',
 'TaxParcelIdentificationNumber',
 'Neighborhood',
 'PrimaryPropertyType',
 'Outlier',
 'BuildingType',
 'YearsENERGYSTARCertified']

In [30]:
data[list(set(data.columns) - set(data._get_numeric_data()))].head(5)

,LargestPropertyUseType,ComplianceStatus,ThirdLargestPropertyUseType,ListOfAllPropertyUseTypes,SecondLargestPropertyUseType,PropertyName,TaxParcelIdentificationNumber,Neighborhood,PrimaryPropertyType,Outlier,BuildingType,YearsENERGYSTARCertified
0,Hotel,Compliant,None,Hotel,None,Mayflower park hotel,0659000030,DOWNTOWN,Hotel,None,NonResidential,None
1,Hotel,Compliant,Restaurant,"Hotel, Parking, Restaurant",Parking,Paramount Hotel,0659000220,DOWNTOWN,Hotel,None,NonResidential,None
2,Hotel,Compliant,Swimming Pool,Hotel,Parking,5673-The Westin Seattle,0659000475,DOWNTOWN,Hotel,None,NonResidential,None
3,Hotel,Compliant,None,Hotel,None,HOTEL MAX,0659000640,DOWNTOWN,Hotel,High Outlier,NonResidential,None
4,Hotel,Compliant,Swimming Pool,"Hotel, Parking, Swimming Pool",Parking,WARWICK SEATTLE HOTEL (ID8),0659000970,DOWNTOWN,Hotel,None,NonResidential,None


<div class="alert alert-block alert-warning">
    <b>TaxParcelIdentificationNumber</b> is a <b>numeric</b> column instead of <b>categorical</b> column
</div>

<div class="alert alert-block alert-info">
    Now, we are going to verify the data in <b>categorical</b> columns
</div>

In [31]:
for col in list(set(data.columns) - set(data._get_numeric_data())):
    print(col)
    display(data[col].unique())

LargestPropertyUseType


array(['Hotel', 'Police Station', 'Other - Entertainment/Public Assembly',
       'Multifamily Housing', 'Library', 'Fitness Center/Health Club/Gym',
       'Social/Meeting Hall', 'Courthouse', 'Other', 'K-12 School',
       'College/University', 'Automobile Dealership', 'Office',
       'Self-Storage Facility', 'Non-Refrigerated Warehouse',
       'Other - Mall', 'Senior Care Community', 'Medical Office',
       'Retail Store', 'Hospital (General Medical & Surgical)', 'Museum',
       'Repair Services (Vehicle, Shoe, Locksmith, etc)',
       'Other - Lodging/Residential', 'Residence Hall/Dormitory',
       'Other/Specialty Hospital', 'Financial Office',
       'Distribution Center', 'Parking', 'Worship Facility', 'Restaurant',
       'Laboratory', 'Supermarket/Grocery Store', 'Convention Center',
       'Urgent Care/Clinic/Other Outpatient', None, 'Other - Services',
       'Strip Mall', 'Wholesale Club/Supercenter',
       'Refrigerated Warehouse', 'Manufacturing/Industrial Plant',
 

ComplianceStatus


array(['Compliant', 'Error - Correct Default Data', 'Missing Data',
       'Non-Compliant'], dtype=object)

ThirdLargestPropertyUseType


...

array([None, 'High Outlier', 'Low Outlier', 'High outlier', 'Low outlier',
       nan], dtype=object)

BuildingType


array(['NonResidential', 'Nonresidential COS', 'Multifamily MR (5-9)',
       'SPS-District K-12', 'Campus', 'Multifamily LR (1-4)',
       'Multifamily HR (10+)', 'Nonresidential WA'], dtype=object)

YearsENERGYSTARCertified


array([None, '2016', '2014', '2012', '20172015',
       '2017201620152014201320122011201020092008', '201020092007',
       '201720102007', '20162015', '20152014201220112009', '201220092008',
       '201620152014201320112010200920072000', '201620092008',
       '201620142012', '20162015201420122008',
       '2016201520142013201220112010200920082007200620052004',
       '201520142013', '2014201320112010200920082007',
       '2017201620152014201320112008', '201720162015201420132008',
       '20162015201320122010', '2015201320122010', '20162011201020092008',
       '2007', '2010', '20172016201520092008', '2017201320092008', '2015',
       '2017', '2017201120102007', '201720162014201320122011201020092007',
       '201720152014201220102008', '2012, 2010, 2009, 2008', '20092005',
       '2017201520142013', '201620152012', '2012201020072006', '20112009',
       '2009', '201620152014201320092008', '201620152014201320112009',
       '2016201520142013201220112010', '201720162012', '20102008',
   

<div class="alert alert-block alert-success">
    At this point, we get that all <b>categorical</b> columns have more than one type of value
</div>

<div class="alert alert-block alert-info">
    <b>Cleaning</b> data in the features
</div>

In [32]:
data["ComplianceStatus"].replace("Non-Compliant", "Not Compliant", inplace=True)
data["ComplianceStatus"].replace("Missing Data", np.nan, inplace=True)
data["ComplianceStatus"].replace("Error - Correct Default Data", np.nan, inplace=True)


data["Neighborhood"].replace("Ballard", "BALLARD", inplace=True)
data["Neighborhood"].replace("Central", "CENTRAL", inplace=True)
data["Neighborhood"].replace("North", "NORTH", inplace=True)
data["Neighborhood"].replace("Delridge", "DELRIDGE", inplace=True)
data["Neighborhood"].replace("DELRIDGE NEIGHBORHOODS", "DELRIDGE", inplace=True)
data["Neighborhood"].replace("Northwest", "NORTHWEST", inplace=True)


data["PrimaryPropertyType"].replace("Restaurant\n", "Restaurant", inplace=True)
data["PrimaryPropertyType"].replace("Self-Storage Facility\n", "Self-Storage Facility", inplace=True)
data["PrimaryPropertyType"].replace("Distribution Center\n", "Distribution Center", inplace=True)
data["PrimaryPropertyType"].replace("Small- and Mid-Sized Office", "Small and Mid-Sized Office", inplace=True)
data["PrimaryPropertyType"].replace("Supermarket / Grocery Store", "Supermarket/Grocery Store", inplace=True)

<div class="alert alert-block alert-info">
    <b>Changing</b> the type of data <b>Neighborhood</b>
</div>

In [33]:
for col in list(set(data.columns) - set(data._get_numeric_data())):
    if col != "TaxParcelIdentificationNumber":
        data[col] = data[col].astype("category")

In [34]:
df_analysis(data, "data")


Analysis of data dataset
--------------------------------------------------------------------
- Dataset shape:                  3432 rows and 43 columns
- Total of NaN values:            16889
- Percentage of NaN:              11.44 %
- Total of full duplicates rows:  0
- Total of empty rows:            0
- Total of empty columns:         0
- Unique indexes:                 True

- Type object and records by columns      ( memory usage: 1.0+ MB )
--------------------------------------------------------------------


,name,type,records,# NaN,% NaN
0,OSEBuildingID,int64,3432,0,0.00000
13,Latitude,float64,3432,0,0.00000
40,YearBuilt,int64,3432,0,0.00000
27,PropertyName,category,3432,0,0.00000
26,PropertyGFATotal,int64,3432,0,0.00000
25,PropertyGFAParking,int64,3432,0,0.00000
24,PropertyGFABuilding(s),int64,3432,0,0.00000
23,PrimaryPropertyType,category,3432,0,0.00000
1,BuildingType,category,3432,0,0.00000
20,NumberofFloors,int64,3432,0,0.00000


<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">3.4. Filtering information</h4>
</div>

<div class="alert alert-block alert-info">
    Based on the mision, we are going to <b>filter</b> the <b>types of building</b> considering only <b>"non residential" buildings</b><br>
    First of all, we should know in which columns we must make focus
</div>

In [35]:
LIST_PROPERTY_COLS = []

for col in data.columns:
    if "Property" in col or "Building" in col:
        LIST_PROPERTY_COLS.append(col)
        
LIST_PROPERTY_COLS

['OSEBuildingID',
 'BuildingType',
 'LargestPropertyUseType',
 'LargestPropertyUseTypeGFA',
 'ListOfAllPropertyUseTypes',
 'NumberofBuildings',
 'PrimaryPropertyType',
 'PropertyGFABuilding(s)',
 'PropertyGFAParking',
 'PropertyGFATotal',
 'PropertyName',
 'SecondLargestPropertyUseType',
 'SecondLargestPropertyUseTypeGFA',
 'ThirdLargestPropertyUseType',
 'ThirdLargestPropertyUseTypeGFA']

In [36]:
data[LIST_PROPERTY_COLS].head(5)

,OSEBuildingID,BuildingType,LargestPropertyUseType,LargestPropertyUseTypeGFA,ListOfAllPropertyUseTypes,NumberofBuildings,PrimaryPropertyType,PropertyGFABuilding(s),PropertyGFAParking,PropertyGFATotal,PropertyName,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA
0,1,NonResidential,Hotel,88434.0,Hotel,1,Hotel,88434,0,88434,Mayflower park hotel,NaN,NaN,NaN,NaN
1,2,NonResidential,Hotel,83880.0,"Hotel, Parking, Restaurant",1,Hotel,88502,15064,103566,Paramount Hotel,Parking,15064.0,Restaurant,4622.0
2,3,NonResidential,Hotel,756493.0,Hotel,1,Hotel,759392,196718,956110,5673-The Westin Seattle,Parking,100000.0,Swimming Pool,0.0
3,5,NonResidential,Hotel,61320.0,Hotel,1,Hotel,61320,0,61320,HOTEL MAX,NaN,NaN,NaN,NaN
4,8,NonResidential,Hotel,123445.0,"Hotel, Parking, Swimming Pool",1,Hotel,113580,62000,175580,WARWICK SEATTLE HOTEL (ID8),Parking,68009.0,Swimming Pool,0.0


<div class="alert alert-block alert-info">
    Based on <b>the percentage of missing-values</b> or the <b>amount of data</b> in each <b>categorical column</b>, we make focus on the following columns: 
    <ul style="list-style-type: square;">
        <li><b>BuildingType</b></li>
        <li><b>PrimaryPropertyType</b></li>
        <li><b>LargestPropertyUseType</b></li>
    </ul> 
</div>

<div class="alert alert-block alert-info">
    Filtering by <b>BuildingType</b> and <b>"non residential"</b> buildings
</div>

In [37]:
list(data["BuildingType"].unique())

['NonResidential',
 'Nonresidential COS',
 'Multifamily MR (5-9)',
 'SPS-District K-12',
 'Campus',
 'Multifamily LR (1-4)',
 'Multifamily HR (10+)',
 'Nonresidential WA']

In [38]:
data = pd.DataFrame(data[~data["BuildingType"].str.contains("family", case=False, na=False).any(level=0)])

In [39]:
df_analysis(data, "data", type_analysis="summarized")


Analysis of data dataset
--------------------------------------------------------------------
- Dataset shape:                  1698 rows and 43 columns
- Total of NaN values:            8333
- Percentage of NaN:              11.41 %
- Total of full duplicates rows:  0
- Total of empty rows:            0
- Total of empty columns:         0
- Unique indexes:                 True




<div class="alert alert-block alert-info">
    Filtering by <b>PrimaryPropertyType</b> and <b>"non residential"</b> buildings
</div>

In [40]:
list(data["PrimaryPropertyType"].unique())

['Hotel',
 'Other',
 'Mixed Use Property',
 'K-12 School',
 'University',
 'Small and Mid-Sized Office',
 'Self-Storage Facility',
 'Warehouse',
 'Large Office',
 'Senior Care Community',
 'Medical Office',
 'Retail Store',
 'Hospital',
 'Residence Hall',
 'Distribution Center',
 'Worship Facility',
 'Supermarket/Grocery Store',
 'Laboratory',
 'Refrigerated Warehouse',
 'Low-Rise Multifamily',
 'Restaurant',
 'Non-Refrigerated Warehouse',
 'Office']

In [41]:
data = data[~data["PrimaryPropertyType"].isin(["Residence Hall", "Residence Hall/Dormitory", "Low-Rise Multifamily"])]

In [42]:
df_analysis(data, "data", type_analysis="summarized")


Analysis of data dataset
--------------------------------------------------------------------
- Dataset shape:                  1674 rows and 43 columns
- Total of NaN values:            8202
- Percentage of NaN:              11.39 %
- Total of full duplicates rows:  0
- Total of empty rows:            0
- Total of empty columns:         0
- Unique indexes:                 True




<div class="alert alert-block alert-info">
    Filtering by <b>LargestPropertyUseType</b> and <b>"non residential"</b> buildings
</div>

In [43]:
list(data["LargestPropertyUseType"].unique())

['Hotel',
 'Police Station',
 'Other - Entertainment/Public Assembly',
 'Library',
 'Fitness Center/Health Club/Gym',
 'Social/Meeting Hall',
 'Courthouse',
 'Other',
 'K-12 School',
 'College/University',
 'Automobile Dealership',
 'Office',
 'Self-Storage Facility',
 'Non-Refrigerated Warehouse',
 'Other - Mall',
 'Senior Care Community',
 'Medical Office',
 'Retail Store',
 'Hospital (General Medical & Surgical)',
 'Museum',
 'Repair Services (Vehicle, Shoe, Locksmith, etc)',
 'Other - Lodging/Residential',
 'Other/Specialty Hospital',
 'Financial Office',
 'Distribution Center',
 'Parking',
 'Multifamily Housing',
 'Worship Facility',
 'Restaurant',
 'Laboratory',
 'Supermarket/Grocery Store',
 'Convention Center',
 'Urgent Care/Clinic/Other Outpatient',
 nan,
 'Other - Services',
 'Strip Mall',
 'Wholesale Club/Supercenter',
 'Refrigerated Warehouse',
 'Manufacturing/Industrial Plant',
 'Other - Recreation',
 'Lifestyle Center',
 'Other - Public Services',
 'Data Center',
 'Other 

In [44]:
data = data[~data["LargestPropertyUseType"].isin(["Other - Lodging/Residential", "Multifamily Housing", "Residence Hall/Dormitory"])]

In [45]:
df_analysis(data, "data")


Analysis of data dataset
--------------------------------------------------------------------
- Dataset shape:                  1659 rows and 43 columns
- Total of NaN values:            8144
- Percentage of NaN:              11.42 %
- Total of full duplicates rows:  0
- Total of empty rows:            0
- Total of empty columns:         0
- Unique indexes:                 True

- Type object and records by columns      ( memory usage: 624.0+ KB )
--------------------------------------------------------------------


,name,type,records,# NaN,% NaN
0,OSEBuildingID,int64,1659,0,0.00000
15,Longitude,float64,1659,0,0.00000
40,YearBuilt,int64,1659,0,0.00000
37,TaxParcelIdentificationNumber,object,1659,0,0.00000
27,PropertyName,category,1659,0,0.00000
26,PropertyGFATotal,int64,1659,0,0.00000
25,PropertyGFAParking,int64,1659,0,0.00000
24,PropertyGFABuilding(s),int64,1659,0,0.00000
23,PrimaryPropertyType,category,1659,0,0.00000
1,BuildingType,category,1659,0,0.00000


<div class="alert alert-block alert-success">
    At this point, we have <b>reduced the dataset</b> shape from <b>3432</b> to <b>1659</b>
</div>

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">3.5. Selecting the features</h4>
</div>

<div class="alert alert-block alert-info">
    Based on the <b>official site of Seattle Open Data</b>, we can see the dataset <b>metadata</b> to know all features
    
    <ul style="list-style-type: square;">
        <li><b>2015: https://data.seattle.gov/dataset/2015-Building-Energy-Benchmarking/h7rm-fz6m</b></li>
        <li><b>2016: https://data.seattle.gov/dataset/2016-Building-Energy-Benchmarking/2bpz-gwpy</b></li>
    </ul>  
    
    On the other hand, we get the means of <b>Energy Star Score</b>
    <ul style="list-style-type: square;">
        <li><b>Energy Star Score: https://www.energystar.gov/buildings/benchmark/analyze_benchmarking_results</b></li>
    </ul>  
       
    The <b>target variables</b> are:
    <ul style="list-style-type: square;">
        <li><b>SiteEnergyUse(kBtu):</b>The annual amount of energy consumed by the property from all sources of energy.</li>
        <li><b>GHGEmissions(MetricTonsCO2e):</b>The total amount of greenhouse gas emissions, including carbon dioxide, methane, and nitrous oxide gases released into the atmosphere as a result of energy consumption at the property, measured in metric tons of carbon dioxide equivalent. This calculation uses a GHG emissions factor from Seattle CIty Light's portfolio of generating resources. This uses Seattle City Light's 2015 emissions factor of 52.44 lbs CO2e/MWh. Enwave steam factor = 170.17 lbs CO2e/MMBtu. Gas factor sourced from EPA Portfolio Manager = 53.11 kg CO2e/MBtu.</li>
    </ul>  
</div>

In [46]:
TARGET_VARIABLES = [
    "SiteEnergyUse(kBtu)", "GHGEmissions(MetricTonsCO2e)"
]

In [47]:
OTHER_VARIABLE = [
    "ENERGYSTARScore"
]

In [48]:
DECLARATIVE_DATA = [
    "BuildingType", "ComplianceStatus", "PrimaryPropertyType", "LargestPropertyUseType", "NumberofBuildings", "NumberofFloors",
    "LargestPropertyUseTypeGFA",  "PropertyGFABuilding(s)", "PropertyGFAParking", "PropertyGFATotal",  
     "DataYear", "YearBuilt", "Neighborhood", "PropertyName"     
]

In [49]:
CONSUMPTION_VARIABLES = [
    "Electricity(kBtu)", "NaturalGas(kBtu)", "SiteEUI(kBtu/sf)", "SourceEUI(kBtu/sf)", "SteamUse(kBtu)", "OtherFuelUse(kBtu)"   
]

<div class="alert alert-block alert-success">
    <b>df_bebcs: Building Energy Benchmarking City of Seattled</b>
</div>

In [50]:
df_bebcs = data[DECLARATIVE_DATA + CONSUMPTION_VARIABLES + OTHER_VARIABLE + TARGET_VARIABLES].copy()

In [51]:
df_analysis(df_bebcs, "df_bebcs", type_analysis="complete")


Analysis of df_bebcs dataset
--------------------------------------------------------------------
- Dataset shape:                  1659 rows and 23 columns
- Total of NaN values:            709
- Percentage of NaN:              1.86 %
- Total of full duplicates rows:  0
- Total of empty rows:            0
- Total of empty columns:         0
- Unique indexes:                 True

- Type object and records by columns      ( memory usage: 469.3 KB )
--------------------------------------------------------------------


,name,type,records,unique,# NaN,% NaN,mean,min,25%,50%,75%,max,std
0,BuildingType,category,1659,5,0,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,PropertyGFABuilding(s),int64,1659,1575,0,0.00000,105901.51537,3636.00000,28382.50000,47560.00000,94606.50000,9320156.00000,284284.82878
13,PropertyName,category,1659,1655,0,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Neighborhood,category,1659,13,0,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,DataYear,int64,1659,2,0,0.00000,2015.98192,2015.00000,2016.00000,2016.00000,2016.00000,2016.00000,0.13329
9,PropertyGFATotal,int64,1659,1580,0,0.00000,119185.34117,11285.00000,29408.50000,49680.00000,105550.00000,9320156.00000,297786.00260
8,PropertyGFAParking,int64,1659,325,0,0.00000,13283.82580,0.00000,0.00000,0.00000,0.00000,512608.00000,43625.30559
11,YearBuilt,int64,1659,113,0,0.00000,1961.69198,1900.00000,1930.00000,1965.00000,1989.00000,2015.00000,32.64154
5,NumberofFloors,int64,1659,44,0,0.00000,4.08861,0.00000,1.00000,2.00000,4.00000,99.00000,6.52489
4,NumberofBuildings,int64,1659,16,0,0.00000,1.17058,0.00000,1.00000,1.00000,1.00000,111.00000,2.93955


<div class="alert alert-block alert-success">
    At this point, <b>we can notice</b> the following:<br/>
        
    <ul style="list-style-type: square;">
        <li>There are some <b>negatives values</b></li>
        <li>There are <b>NumberofBuildings</b> and <b>NumberofFloors</b> with <b>0</b></li>
        <li><b>Each target variable </b>has <b>an observation</b> with <b>missing-values</b></li>
        <li>We can infer the age of the building if we work with the fields <b>YearBuilt</b> and <b>DataYear</b></li>
    </ul>  
    <b>To Do</b><br/>
    Due to that, we are going to consider the following <b>hypothesis</b>. 
    <ul style="list-style-type: square;">
        <li>In <b>this case</b> it is not possible to have <b>negative values</b>. So, we are going to <b>format</b> all dataframe with <b>absolute values</b></li>
        <li>We are going to <b>delete</b> the <b>target</b> variable observations with <b>missing-values</b></li>
        <li>There is not <b>NumberofBuildings</b> and <b>NumberofFloors</b> with value of <b>0</b>. We are going to <b>find</b> this information on <b>internet</b>.<br> 
            <b>If it is not possible</b>. So, we are going to <b>replace</b> these values <b>by 1</b></li>
    </ul>     
</div>

<div class="alert alert-block alert-info">
    <b>Formatting</b> all dataframe with <b>absolute values</b>.
</div>

In [52]:
for col in set(df_bebcs._get_numeric_data()):
    df_bebcs[col] = df_bebcs[col].abs()

<div class="alert alert-block alert-info">
    We are going to <b>delete</b> the <b>target</b> variable observations with <b>missing-values</b>
</div>

In [55]:
df_bebcs = pd.DataFrame(df_bebcs[df_bebcs[TARGET_VARIABLES].notnull().all(axis=1)]).reset_index(drop=True)

In [56]:
df_analysis(df_bebcs, "df_bebcs", type_analysis="complete")


Analysis of df_bebcs dataset
--------------------------------------------------------------------
- Dataset shape:                  1658 rows and 23 columns
- Total of NaN values:            700
- Percentage of NaN:              1.84 %
- Total of full duplicates rows:  0
- Total of empty rows:            0
- Total of empty columns:         0
- Unique indexes:                 True

- Type object and records by columns      ( memory usage: 391.8 KB )
--------------------------------------------------------------------


,name,type,records,unique,# NaN,% NaN,mean,min,25%,50%,75%,max,std
0,BuildingType,category,1658,5,0,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,DataYear,int64,1658,2,0,0.00000,2015.98251,2015.00000,2016.00000,2016.00000,2016.00000,2016.00000,0.13113
21,SiteEnergyUse(kBtu),float64,1658,1643,0,0.00000,8575774.22980,0.00000,1217659.84375,2554947.25000,6917428.87500,873923712.00000,30553306.20725
18,SteamUse(kBtu),float64,1658,115,0,0.00000,526825.75363,0.00000,0.00000,0.00000,0.00000,134943456.00000,5550927.45242
17,SourceEUI(kBtu/sf),float64,1658,1274,0,0.00000,176.87503,0.00000,76.40000,131.40000,204.07500,2620.00000,187.83787
16,SiteEUI(kBtu/sf),float64,1658,978,0,0.00000,72.59789,0.00000,33.70000,51.75000,81.65000,834.40002,75.47324
15,NaturalGas(kBtu),float64,1658,1180,0,0.00000,2015022.59230,0.00000,0.00000,483518.00000,1520000.25000,297909000.00000,9590328.21558
14,Electricity(kBtu),float64,1658,1647,0,0.00000,5922510.82635,0.00000,720540.75000,1616586.00000,4890859.75000,657074389.00000,20926670.21669
13,PropertyName,category,1658,1654,0,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Neighborhood,category,1658,13,0,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX</h3>
</div>

<div class="alert alert-block alert-danger">
    <b>Flag Position</b>
</div>

In [53]:
CCCCCCCCCCCCC

NameError: name 'CCCCCCCCCCCCC' is not defined

In [ ]:
for col in list(set(data.columns) - set(data._get_numeric_data())):
    if col != "TaxParcelIdentificationNumber":
        print(data.groupby(col)["OSEBuildingID"].count().sort_values(ascending=False))
        print("________________________________________________________________________")

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX</h3>
</div>

<div class="alert alert-block alert-danger">
    <b>Flag Position</b>
</div>

In [ ]:
XXXXXXXXXXXXXX

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">3.4. Selecting the features</h4>
</div>

<div class="alert alert-block alert-info">
    Based on the <b>official site of Seattle Open Data</b>, we can see the dataset <b>metadata</b> to know all features
    
    <ul style="list-style-type: square;">
        <li><b>2015: https://data.seattle.gov/dataset/2015-Building-Energy-Benchmarking/h7rm-fz6m</b></li>
        <li><b>2016: https://data.seattle.gov/dataset/2016-Building-Energy-Benchmarking/2bpz-gwpy</b></li>
    </ul>  
    
    On the other hand, we get the means of <b>Energy Star Score</b>
    <ul style="list-style-type: square;">
        <li><b>Energy Star Score: https://www.energystar.gov/buildings/benchmark/analyze_benchmarking_results</b></li>
    </ul>  
       
    The <b>target variables</b> are:
    <ul style="list-style-type: square;">
        <li><b>SiteEnergyUse(kBtu):</b>The annual amount of energy consumed by the property from all sources of energy.</li>
        <li><b>GHGEmissions(MetricTonsCO2e):</b>The total amount of greenhouse gas emissions, including carbon dioxide, methane, and nitrous oxide gases released into the atmosphere as a result of energy consumption at the property, measured in metric tons of carbon dioxide equivalent. This calculation uses a GHG emissions factor from Seattle CIty Light's portfolio of generating resources. This uses Seattle City Light's 2015 emissions factor of 52.44 lbs CO2e/MWh. Enwave steam factor = 170.17 lbs CO2e/MMBtu. Gas factor sourced from EPA Portfolio Manager = 53.11 kg CO2e/MBtu.</li>
        <li><b>ENERGYSTARScore:</b>An EPA calculated 1-100 rating that assesses a property’s overall energy performance, based on national data to control for differences among climate, building uses, and operations. A score of 50 represents the national median.</li>
    </ul>  
</div>

In [ ]:
TARGET_VARIABLES = [
    "SiteEnergyUse(kBtu)", "GHGEmissions(MetricTonsCO2e)", "ENERGYSTARScore"
]

In [ ]:
GENERAL_VARIABLES = [
    "OSEBuildingID", "DataYear", "PrimaryPropertyType", "renamed_PrimaryPropertyType", 
    "YearBuilt", "PropertyGFATotal", "NumberofFloors", "NumberofBuildings", 
    "ZipCode", "Neighborhood", 
]

In [ ]:
OTHERS_VARIABLES = [
    "Electricity(kBtu)", "NaturalGas(kBtu)", "SiteEUI(kBtu/sf)", "SourceEUI(kBtu/sf)", "SteamUse(kBtu)", 
]

<div class="alert alert-block alert-success">
    <b>df_bebcs: Building Energy Benchmarking City of Seattled</b>
</div>

In [ ]:
df_bebcs = data[GENERAL_VARIABLES + TARGET_VARIABLES + OTHERS_VARIABLES].copy()

In [ ]:
df_analysis(df_bebcs, "df_bebcs", type_analysis="complete")

<div class="alert alert-block alert-success">
    <b>Hypothesis</b><br/>
    At this point, we can notice the following:
        
    <ul style="list-style-type: square;">
        <li>There are some <b>negatives values</b></li>
        <li>There are <b>NumberofBuildings</b> and <b>NumberofFloors</b> with <b>0</b></li>
        <li>We can infer the age of the building if we work with the fields <b>YearBuilt</b> and <b>DataYear</b></li>
    </ul>  
    
    Due to that, we are going to consider the following <b>hypothesis</b>. 
    <ul style="list-style-type: square;">
        <li>In <b>this case</b> it is not possible to have <b>negative values</b>. So, we are going to <b>format</b> all dataframe with <b>absolute values</b>.</li>
        <li>There is not <b>NumberofBuildings</b> and <b>NumberofFloors</b> with value of <b>0</b>. So, we are going to <b>replace</b> these values <b>by 1</b></li>
    </ul>     
</div>

<div class="alert alert-block alert-info">
    <b>Formatting</b> all dataframe with <b>absolute values</b>.
</div>

In [ ]:
for col in set(df_bebcs._get_numeric_data()):
    df_bebcs[col] = df_bebcs[col].abs()

<div class="alert alert-block alert-info">
    <b>Replacing by 1</b>, value of 0 in <b>(NumberofBuildings = 0)</b> and <b>(NumberofFloors = 0)</b> 
</div>

In [ ]:
df_bebcs.loc[df_bebcs["NumberofBuildings"] == 0, "NumberofBuildings"] = 1
df_bebcs.loc[df_bebcs["NumberofFloors"] == 0, "NumberofFloors"] = 1

<div class="alert alert-block alert-info">
    Calculating the <b>building age</b> 
</div>

In [ ]:
df_bebcs["BuildingAge"] = df_bebcs["DataYear"] - df_bebcs["YearBuilt"]

<div style="background-color: #506AB9;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">4. Treating Outliers</h3>
</div>

<div class="alert alert-block alert-info">
    We are going to <b>treat </b> outliers only for variables <b>that are not targets</b>. Also, we are going to consider <b>PrimaryPropertyType</b> to treat the outliers.
    <ul style="list-style-type: square;">
        <li><b>Electricity(kBtu)</b></li>
        <li><b>NaturalGas(kBtu)</b></li>
        <li><b>SiteEUI(kBtu/sf)</b></li>
        <li><b>SourceEUI(kBtu/sf)</b></li>
        <li><b>SteamUse(kBtu)</b></li>
    </ul>  
</div>

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">4.1. Treating Outliers with Z Score</h4>
</div>

<div class="alert alert-block alert-info">
    Deleting <b>outliers</b> with <b>Z Score</b>
</div>

In [ ]:
df_without_outliers = pd.DataFrame()

# List de features to excluded because they don't need to be treated cause they are targets or general information
LIST_FEATURES_TO_BE_EXCLUDED = GENERAL_VARIABLES + TARGET_VARIABLES + ["BuildingAge"]

for property_type in df_bebcs["PrimaryPropertyType"].unique():
    
    # for each property type, we are going to define the max and min threshold to replace outliers
    df_temp_outliers = df_bebcs[df_bebcs["PrimaryPropertyType"] == property_type].copy()
    
    for col in set(df_temp_outliers._get_numeric_data()):
        
        if col not in LIST_FEATURES_TO_BE_EXCLUDED:
            # Defining the Z Score
            df_temp_outliers[col + "_zscore"] = (df_temp_outliers[col] - df_temp_outliers[col].mean()) / df_temp_outliers[col].std()
            df_temp_outliers.loc[(df_temp_outliers[col + "_zscore"] > 3) | (df_temp_outliers[col + "_zscore"] < -3), col] = np.nan

            del df_temp_outliers[col + "_zscore"]
        
    df_without_outliers = pd.concat([df_without_outliers, df_temp_outliers])

In [ ]:
df_analysis(df_without_outliers, "df_without_outliers", type_analysis="complete")

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">4.2. Comparison of Outliers treatment</h4>
</div>

<div class="alert alert-block alert-info">
    Identifying the <b>PrimaryPropertyType</b> with more records to plot the comparison with <b>Z Score result</b>
</div>

In [ ]:
df_bebcs.groupby("renamed_PrimaryPropertyType")["OSEBuildingID"].count().sort_values(ascending=False).head(3)

<div class="alert alert-block alert-info">
    <b>Plotting</b> the comparison between data before and after treating <b>outliers</b> with <b>Z Score</b>
</div>

In [ ]:
LIST_FEATURES_TO_PLOTT = ["Electricity(kBtu)", "NaturalGas(kBtu)"]
property_type = "Large Office"

df_temp = df_bebcs[df_bebcs["PrimaryPropertyType"] == property_type].copy()
df_temp_by_zscore = df_without_outliers[df_without_outliers["PrimaryPropertyType"] == property_type].copy()

for col in LIST_FEATURES_TO_PLOTT:

    # Creating a figure composed of two matplotlib.Axes objects (ax_box and ax_hist)
    fig, ((ax_box, ax_box_zscore), (ax_hist, ax_hist_zscore)) = \
                                                        plt.subplots(nrows=2, ncols=2, figsize=[14,8], sharex=True, gridspec_kw={"height_ratios": (.15, .85)})

    # Title for the complete RR078466118RUfigure
    fig.suptitle((col.capitalize() + " in " + property_type), fontsize="x-large", fontweight="bold", size=20)

    # Assigning a graph to each ax
    ax_box.title.set_text("Before treating outliers")
    ax_box_zscore.title.set_text("After zscore")

    sns.boxplot(data=df_temp, x=col, ax=ax_box)
    sns.boxplot(data=df_temp_by_zscore, x=col, ax=ax_box_zscore)

    sns.histplot(data=df_temp, x=col, kde=True, ax=ax_hist)
    sns.histplot(data=df_temp_by_zscore, x=col, kde=True, ax=ax_hist_zscore)

    # Remove x axis name for the boxplot
    ax_box.set(xlabel="")
    ax_box.set(yticks=[])
    ax_box_zscore.set(xlabel="")
    ax_box_zscore.set(yticks=[])

    plt.tight_layout()
    plt.savefig("img/outliers-" + (col[0:-5].capitalize()) + "-in-" + property_type + ".png")
    sns.despine(fig)

<div class="alert alert-block alert-success">
    Both percentiles and distributions appear similar
</div>

<div style="background-color: #506AB9;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">5. Exploratory Data Analysis</h3>
</div>
<br>
<div class="alert alert-block alert-info">
    We are just going to make an <b>Exploraty Data Analysis</b> based on the following<b>target variables</b>
    <ul style="list-style-type: square;">
        <li><b>SiteEnergyUse(kBtu)</b></li>
        <li><b>GHGEmissions(MetricTonsCO2e)</b></li>
    </ul>  
</div>

In [ ]:
df_analysis(df_without_outliers, "df_without_outliers", type_analysis="complete")

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">5.1. Initial analysis</h4>
</div>

<div style="background-color: #6D83C5;" >
    <h5 style="margin: auto; padding: 20px; color:#fff; ">5.1.1 Primary Property type distribution</h5>
</div>

In [ ]:
data_piechart = df_without_outliers["renamed_PrimaryPropertyType"].sort_values(ascending=False).value_counts().values
labels_piechart = df_without_outliers["renamed_PrimaryPropertyType"].sort_values(ascending=False).value_counts().index.tolist()
textprops = dict(size=16, color="gray")
colors = ["#5975A4", "#CC8963", "#5F9E6E", "#AD5B5E", "#857AAB", "#7D6D5F", "#D095BF", "#EAEAF2", "#C1B37F"]
explode = (0, 0, 0, 0, 0, 0, 0, 0, 0)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=[15,8])

fig.suptitle("Primary Property type distribution", size=25)

for lb, data, color in zip(labels_piechart, data_piechart, colors):
    ax2.bar(lb, data, label=lb, color=color)
for index, data in enumerate(data_piechart):
    plt.text(x=index, y=data+1 , s=f"{data}", horizontalalignment="center", fontdict=dict(fontsize=16, color="gray"))
ax2.set_title("by quantity (#)", fontsize=14)
ax2.axis('off')

ax1.pie(x=data_piechart, explode=explode, autopct="%1.1f%%", pctdistance=1.10, 
        textprops=textprops, colors=[color for color in colors], startangle=0)
ax1.set_title("by percentage (%)", fontsize=14)

fig.legend(labels_piechart, loc="center", ncol=5, fontsize=14, bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True)

plt.tight_layout()
plt.savefig("img/primary-property-type-distribution.png")
plt.show()

<div style="background-color: #6D83C5;" >
    <h5 style="margin: auto; padding: 20px; color:#fff; ">5.1.2  Comparison of the targets variables in each Property type</h5>
</div>

In [ ]:
for var in TARGET_VARIABLES:
    if var != "ENERGYSTARScore":
        plt.figure(figsize=[12,10])
        sns.boxplot(x=df_without_outliers[var], y=df_without_outliers["renamed_PrimaryPropertyType"], orient="h", showmeans=True, 
                    meanprops={"markerfacecolor":"white", "markeredgecolor":"blue"})
        plt.title(var + " in all categories", size=20)
        plt.ylabel("Property Type", size=14)
        plt.xlabel("Quantity of " + var, size=14)
        plt.tight_layout()
        plt.savefig("img/comparison-of-" + var + "-in-each-category.png")
        plt.show()

<div class="alert alert-block alert-success">
    <b>Observations / Conclusions</b><br/><br/>
    Building related to <b>Health</b> have a high <b>SiteEnergyUse(kBtu)</b> and <b>GHGEmissions(MetricTonsCO2e)</b>
</div>

<div style="background-color: #6D83C5;" >
    <h5 style="margin: auto; padding: 20px; color:#fff; ">5.1.3  Comparison by Years</h5>
</div>

In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 10))
sns.barplot(x="renamed_PrimaryPropertyType", 
            y="SiteEnergyUse(kBtu)", 
            hue="DataYear", 
            data=df_without_outliers)
plt.legend(bbox_to_anchor=(1.004, 1), borderaxespad=0, title="Years")
plt.ylabel("Site Energy Use (kBtu)", size=14)
plt.xlabel("Property Type", size=14)
plt.title("Site Energy Use by year in each property type", size=18)
plt.savefig("img\grouped-SiteEnergyUse-by-year.png")

In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 10))
sns.barplot(x="renamed_PrimaryPropertyType", 
            y="GHGEmissions(MetricTonsCO2e)", 
            hue="DataYear", 
            data=df_without_outliers)
plt.legend(bbox_to_anchor=(1.004, 1), borderaxespad=0, title="Years")
plt.ylabel("Total GHG Emissions (MetricTonsCO2e)", size=14)
plt.xlabel("Property Type", size=14)
plt.title("Total GHG Emissions by year in each property type", size=18)
plt.savefig("img\grouped-GHGEmissions-by-year.png")

<div class="alert alert-block alert-success">
    <b>Observations / Conclusions</b><br/><br/>
    Building related to <b>Health</b> have a high <b>SiteEnergyUse(kBtu)</b> and <b>GHGEmissions(MetricTonsCO2e)</b>
</div>

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">5.2. Univariate analysis</h4>
</div>
<br>
<div class="alert alert-block alert-info">
    We are are going to plot the <b>not missing-values</b> of the following target values
    <ul style="list-style-type: square;">
        <li><b>SiteEnergyUse(kBtu)</b></li>
        <li><b>GHGEmissions(MetricTonsCO2e)</b></li>
    </ul>  
</div>

In [ ]:
# Filtering by non missing-values to can plot the QQplot
df_temp = pd.DataFrame(df_without_outliers[df_without_outliers[["SiteEnergyUse(kBtu)", "GHGEmissions(MetricTonsCO2e)"]].notnull().all(axis=1)])
        
for var in TARGET_VARIABLES:
    
    if var != "ENERGYSTARScore":

        fig = plt.figure(constrained_layout=True, figsize=[15,7])
        fig.suptitle(var, size=25, fontweight="bold", y=1.05)
        spec = fig.add_gridspec(nrows=2, ncols=2, width_ratios=[1, 1], height_ratios=[1, 5])

        ax_box = fig.add_subplot(spec[0, 0])
        ax_hist = fig.add_subplot(spec[1, 0])
        ax_qqplot = fig.add_subplot(spec[:, 1])

        # Boxplot
        boxplot = sns.boxplot(data=df_temp, x=df_temp[var], showmeans=True, ax=ax_box,
                              meanprops={"markerfacecolor":"white", "markeredgecolor":"blue"})
        ax_box.set(xlabel="", xticks=[])
        ax_box.set(yticks=[])
        boxplot.set_title("")

        # Histplot
        sns.histplot(data=df_temp, x=df_temp[var], bins=75,  kde=True,  ax=ax_hist)
        if var == "SiteEnergyUse(kBtu)":
            ax_hist.set(xlabel="kBtu")
        else:
            ax_hist.set(xlabel="MetricTonsCO2e")

        # QQplot
        qqplot(df_temp[var], line='s', ax=ax_qqplot)

        plt.savefig("img/qqplot-of-" + var + ".png")
        plt.show()

        print("\nMaking the Normality Test")
        print("---------------------------------")
        normality_test(df_temp[[var]])
        print("\n")

In [ ]:
pd.reset_option("display.max_colwidth") # reset width of showing cols

<div class="alert alert-block alert-success">
    <b>Observations / Conclusions</b>
    <ul style="list-style-type: square;">
        <li><b>SiteEnergyUse(kBtu)</b> and <b>GHGEmissions(MetricTonsCO2e)</b> do not have a <b>Normal distribution</b></li>
        <li>Based on the distribution plot, we notice that <b>all values are very close</b> what remains to be a problem</li>
    </ul>  
</div>

<div style="background-color: #6D83C5;" >
    <h5 style="margin: auto; padding: 20px; color:#fff; ">5.2.1 Log Normalization target variables</h5>
</div>
<br>
<div class="alert alert-block alert-info">
    Based on the <b>Univariate analysis</b> where we note that <b>SiteEnergyUse(kBtu)</b> and <b>GHGEmissions(MetricTonsCO2e)</b> do not have a <b>Normal distribution</b>
    and <b>all their values are very close</b> so, we are going to aplly the <b>Natural Logarithmic transformation</b> in these variables to get a <b>better Normal distribution</b> 
    of these values
</div>

In [ ]:
df_bebcs_log = df_without_outliers.copy()

<div class="alert alert-block alert-info">
    <b>To avoid missing-values</b>, we are going to apply <b>np.log(variable + 1)</b>
</div>

In [ ]:
df_bebcs_log["SiteEnergyUse(kBtu)_log"] = np.log(df_bebcs_log["SiteEnergyUse(kBtu)"] + 1)
df_bebcs_log["GHGEmissions(MetricTonsCO2e)_log"] = np.log(df_bebcs_log["GHGEmissions(MetricTonsCO2e)"] + 1)

In [ ]:
# Filtering by non missing-values to can plot the QQplot
df_temp = pd.DataFrame(df_bebcs_log[df_bebcs_log[["SiteEnergyUse(kBtu)", "SiteEnergyUse(kBtu)_log", "GHGEmissions(MetricTonsCO2e)", "GHGEmissions(MetricTonsCO2e)_log"]].notnull().all(axis=1)])
        
for var in ["SiteEnergyUse(kBtu)", "SiteEnergyUse(kBtu)_log", "GHGEmissions(MetricTonsCO2e)", "GHGEmissions(MetricTonsCO2e)_log"]:
    
    fig = plt.figure(constrained_layout=True, figsize=[14,7])
    fig.suptitle(var, size=25, fontweight="bold", y=1.05)
    spec = fig.add_gridspec(nrows=2, ncols=2, width_ratios=[1, 1], height_ratios=[1, 5])

    ax_box = fig.add_subplot(spec[0, 0])
    ax_hist = fig.add_subplot(spec[1, 0])
    ax_qqplot = fig.add_subplot(spec[:, 1])

    # Boxplot
    boxplot = sns.boxplot(data=df_temp, x=df_temp[var], showmeans=True, ax=ax_box,
                          meanprops={"markerfacecolor":"white", "markeredgecolor":"blue"})
    ax_box.set(xlabel="", xticks=[])
    ax_box.set(yticks=[])
    boxplot.set_title("")

    # Histplot
    sns.histplot(data=df_temp, x=df_temp[var], bins=75,  kde=True,  ax=ax_hist)
    if "SiteEnergyUse" in var:
        ax_hist.set(xlabel="kBtu")
    else:
        ax_hist.set(xlabel="MetricTonsCO2e")

    # QQplot
    qqplot(df_temp[var], line='s', ax=ax_qqplot)

    plt.savefig("img/qqplot-of-" + var + ".png")
    plt.show()

    print("\nMaking the Normality Test")
    print("---------------------------------")
    normality_test(df_temp[[var]])
    print("\n")

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">5.3 Multivariate Analysis</h4>
</div>
<br>
<div style="background-color: #6D83C5;" >
    <h5 style="margin: auto; padding: 20px; color:#fff; ">5.3.1 Correlation Matrix</h5>
</div>

In [ ]:
df_analysis(df_bebcs_log, "df_bebcs_log", type_analysis="complete")

In [ ]:
df_correlation_matrix = pd.DataFrame(df_bebcs_log.corr()).round(2)

In [ ]:
mask = np.zeros_like(df_correlation_matrix, dtype=bool)
mask[np.triu_indices_from(mask)] = True 

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Pearson correlation between all features", size=20)

ax = sns.heatmap(df_correlation_matrix, mask=mask, vmin=-1, cmap="mako_r",
                annot=True, annot_kws={"size": 14})

plt.xticks(rotation=25, size=14, horizontalalignment="right")
plt.yticks(rotation=0, size=14)
plt.tight_layout()
plt.savefig("img/pearson-correlation-between-features.png")
plt.show()

<div class="alert alert-block alert-success">
    <b>Observations / Conclusions</b>
    
        <ul style="list-style-type: square;">
            <li><b>Energy-kcal_100g</b> and <b>sugars_100g</b> have a strong direct relation with <b>nutriscore_score</b></li>
        </ul>               
</div>

<div style="background-color: #6D83C5;" >
    <h4 style="margin: auto; padding: 20px; color:#fff; ">5.4. Principal component analysis - PCA</h4>
</div>
<br>
<div class="alert alert-block alert-info">
    To make the PCA, we are are only going to work with the following features:
    <ul style="list-style-type: square;">
        <li>YearBuilt</li>
        <li>PropertyGFATotal</li>
        <li>NumberofFloors</li>
        <li>NumberofBuildings</li>
        <li>Electricity(kBtu)</li>
        <li>NaturalGas(kBtu)</li>
        <li>SiteEUI(kBtu/sf)</li>
        <li>SourceEUI(kBtu/sf)</li>
        <li>SteamUse(kBtu)</li>
    </ul>  
</div>

In [ ]:
TARGET_VARIABLES_LOG = [
    "SiteEnergyUse(kBtu)_log", "GHGEmissions(MetricTonsCO2e)_log"
]

In [ ]:
PCA_VARIABLES = [
    "YearBuilt", "PropertyGFATotal", "NumberofFloors", "NumberofBuildings", 
    "Electricity(kBtu)", "NaturalGas(kBtu)", "SiteEUI(kBtu/sf)", "SourceEUI(kBtu/sf)", 
    "SteamUse(kBtu)"
]

In [ ]:
OTHERS_VARIABLES = [
    "OSEBuildingID", "DataYear", "PrimaryPropertyType", "renamed_PrimaryPropertyType", 
    "ZipCode", "Neighborhood", 
]

<div class="alert alert-block alert-info">
    Considering only non missing-values
</div>

In [ ]:
df_subset = df_bebcs_log[df_bebcs_log[PCA_VARIABLES + TARGET_VARIABLES_LOG].notnull().all(axis=1)].copy()

In [ ]:
df_analysis(df_subset, "df_subset", type_analysis="complete")

<div class="alert alert-block alert-info">
   Numbers of components to calculate to PCA            
</div>

In [ ]:
n_comp = 6

<div class="alert alert-block alert-info">
    Selecting the features to analyse with PCA
</div>

In [ ]:
X = df_subset.loc[:, PCA_VARIABLES].values
index_values = df_subset.loc[:, PCA_VARIABLES].index
features = df_subset.loc[:, PCA_VARIABLES].columns

<div class="alert alert-block alert-info">
   Data standardization      
</div>

In [ ]:
X_scaled = StandardScaler().fit_transform(X)

<div class="alert alert-block alert-info">
   Principal components determination      
</div>

In [ ]:
pca = PCA(n_components=n_comp).fit(X_scaled)

In [ ]:
df_pca = pd.DataFrame(pca.components_, 
                      index=["PC" + str(i+1) for i in range(n_comp)],
                      columns=features ).T

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Principal components coefficients", size=20)

ax = sns.heatmap(df_pca, vmin=-1, vmax=1, center=0, cmap="mako_r",
                annot=True, annot_kws={"size": 14})

plt.xticks(size=14)
plt.yticks(size=14)
plt.tight_layout()
plt.savefig("img/principal-components-coefficients.png")
plt.show()

<div class="alert alert-block alert-success">
    <b>Observations / Conclusions</b><br>
    
    In this graph we have the influence of each of the variables for each component
    
    <ul style="list-style-type: square;">
        <li><b>PC1</b> has a influence by <b>Electricity(kBtu)</b></li>
        <li><b>PC2</b> has a strong influence by <b>SiteEUI(kBtu/sf)</b> and <b>SourceEUI(kBtu/sf)</b></li>
        <li><b>PC3</b> has a strong influence by <b>NumberofBuildings</b></li>
        <li><b>PC4</b> has a strong influence by <b>NumberofBuildings</b> and <b>YearBuilt</b></li>
        <li><b>PC5</b> has a strong influence by <b>SteamUse(kBtu)</b> and <b>YearBuilt</b></li>
        <li><b>PC6</b> has a strong influence by <b>NaturalGas(kBtu)</b></li>
    </ul>     
</div>

<div class="alert alert-block alert-info">
    Plotting the <b>Scree of eigenvalues</b>
</div>

In [ ]:
scree = pca.explained_variance_ratio_*100

fig = plt.subplots(figsize=(10, 7))
plot_bar = sns.barplot(x=df_pca.columns, y=scree)
for p in plot_bar.patches:
    plot_bar.annotate(format(p.get_height(), ".1f"), (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha="center", va="bottom", xytext=(0, -15), textcoords="offset points", color="white")
plot_line = sns.lineplot(x=df_pca.columns, y=scree.cumsum(), marker="o", color="red")

for x, y in zip(df_pca.columns, scree.cumsum()):
    plot_line.text(x, y, "{:.2f}".format(y), ha="right", va="bottom")

plt.xlabel("Rank of the inertia axis", size=14)
plt.ylabel("% of inertia", size=14)
plt.title("Scree of eigenvalues", size=20)
plt.tight_layout()
plt.savefig("img/scree-of-eigenvalues.png")
plt.show()

<div class="alert alert-block alert-success">
    <b>Observations / Conclusions</b>
    
    <ul style="list-style-type: square;">
        <li>Up to <b>PC6</b>, we describe <b>95%</b> of the variance of the data. So, in this case we are going to take <b>all components</b></li>
    </ul>     
</div>
<div class="alert alert-block alert-info">
    <b>Keeping the 90% of the variance of the components</b>
</div>

In [ ]:
var_threshold=0.9

# Processing the PCA
pca = decomposition.PCA()
pca.fit(X_scaled)

# Getting the explained variance ratio for each principal component
scree = pca.explained_variance_ratio_

# Getting the number of principal components to reach variance thresholds
mask = scree.cumsum() > var_threshold
nb_selected_components = len(scree[~mask]) + 1
print("Number of selected components:", nb_selected_components)

# Compute and displays the actual ratio of explained variance
explained_variance_sum = scree.cumsum()[nb_selected_components-1]
print("Cumulative explained variance:  {:.2f}%".format(explained_variance_sum*100))

# Getting the projection of the data on the first components
X_PCA = pca.transform(X_scaled)[:,:nb_selected_components]

In [ ]:
df_worked = pd.DataFrame(data=X_PCA, columns=["PC1", "PC2", "PC3", "PC4", "PC5", "PC6"], index=index_values)

In [ ]:
df_worked.head(5)

In [ ]:
df_worked = pd.concat([df_worked, df_subset[OTHERS_VARIABLES], df_subset[TARGET_VARIABLES_LOG]], axis=1)

<div class="alert alert-block alert-info">
    Ordering the columns
</div>

In [ ]:
df_worked = df_worked[OTHERS_VARIABLES +
                      list(set(df_worked.columns) - set(OTHERS_VARIABLES + TARGET_VARIABLES_LOG)) +
                      TARGET_VARIABLES_LOG]

In [ ]:
df_worked.head(5)